In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

df = pd.read_csv('NIFTY50_all.csv')
df.drop(['Trades', 'Deliverable Volume', 'Volume', 'Turnover', '%Deliverble','Series'], inplace = True,axis=1)
scaler = StandardScaler()
df[['Prev Close', 'Open', 'High', 'Low', 'Last', 'Close', 'VWAP']] = scaler.fit_transform(df[['Prev Close', 'Open', 'High', 'Low', 'Last', 'Close', 'VWAP']])

In [3]:
symbol_data = df[['Symbol']]
encoder = OneHotEncoder(sparse_output= False)
encoded_data = encoder.fit_transform(symbol_data)
encoded_df = pd.DataFrame(encoded_data, columns=encoder.categories_[0])
df = pd.concat([df, encoded_df], axis=1)
train_percentage = 0.8
train_data = []
test_data = []
for symbol, company_data in df.groupby('Symbol'):
    split_index = int(len(company_data) * train_percentage)
    train_set = company_data[:split_index]
    test_set = company_data[split_index:]
    train_data.append(train_set)
    test_data.append(test_set)
train_data = pd.concat(train_data)
test_data = pd.concat(test_data)
df=df.drop('Symbol', axis=1)
train_data=train_data.drop('Symbol', axis=1)
test_data=test_data.drop('Symbol', axis=1)

train_data['year'] = pd.to_datetime(df.Date).dt.year
train_data.set_index('year', inplace=True)
train_data.sort_index(inplace=True)
test_data['year'] = pd.to_datetime(df.Date).dt.year
test_data.set_index('year', inplace=True)
test_data.sort_index(inplace=True)

X_train= train_data.drop(labels=['Open', 'Date'], axis=1)
X_test= test_data.drop(labels=['Open', 'Date'], axis=1)
y_train = train_data['Open']
y_test = test_data['Open']



xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)

y_pred = xgb_reg.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)


3.3799016436783353
